In [50]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce,api
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [49]:
results = sf.query_all(
    """
    select id,Paciente__r.IdentificationId__pc, padre__c,Paciente__c,Paciente__r.Habilitado__c,Programas_por_Empresas__r.Datos_Empresa__c,datos2__c,  Programas_por_Empresas__c,Programas_por_Empresas__r.Name,  Empleador__c  from Paciente_Programas__c where Id = 'a1Y8c00000C5asnEAB' 
    """
)
derttyuiop = pd.DataFrame(results["records"])
derttyuiop.to_excel(
        "F:/Empresas SF/reportes/aaaaaa.xlsx", index=False
    )


In [51]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(6943, 7)

In [52]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6651, 5)

In [53]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(6949, 9)

fin empresas y comienzo de rescate de pacientes


In [54]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1629735, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [55]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña"        
    }
)
Paciente_Programas_SF.shape

(1194460, 4)

In [56]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1195684, 12)

In [57]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

PermissionError: [Errno 13] Permission denied: 'F:/Empresas SF/reportes/empresas.xlsx'

In [ ]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353


In [40]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [41]:
#pacientes_empresas22.shape

In [42]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [43]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [44]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [25]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "BCI"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [26]:
Pacientes_Archivo.head(10)

,Empresa,Rut Titular,DV,Rut carga,Dv carga,rut1,sexo,Parentesco,Apellido Paterno,Apellido Mateno,...,Nombre,Nombre 2,nombres,naxxxcimiento,Columna3,Columna4,Columna1,nacimiento,email,celular
0,Pesco Seguridad y Servicios S.A.,8650247,k,NaN,NaN,8650247-k,M,Titular,Villalobos,Riquelme,...,Marco,Antonio,Marco Antonio,1964-02-26 00:00:00,26,2,1964,1964-02-26,bomberovillalobos@hotmail.com,948941996.0
1,Pesco Seguridad y Servicios S.A.,8849139,4,NaN,NaN,8849139-4,M,Titular,Barahona,Lobos,...,Patricio,Miguel,Patricio Miguel,1963-05-03 00:00:00,3,5,1963,1963-05-03,barahona.patricio@gmail.com,957288339.0
2,Pesco Seguridad y Servicios S.A.,9120472,k,NaN,NaN,9120472-k,M,Titular,Soza,Romero,...,Jose,Alejandro,Jose Alejandro,1962-01-10 00:00:00,10,1,1962,1962-01-10,josesozar@gmail.com,NaN
3,Pesco Seguridad y Servicios S.A.,9616064,k,NaN,NaN,9616064-k,M,Titular,Perez,Anker,...,Jose,Joaquin,Jose Joaquin,1969-07-08 00:00:00,8,7,1969,1969-07-08,jperezanker@gmail.com,987740031.0
4,Pesco Seguridad y Servicios S.A.,9906103,0,NaN,NaN,9906103-0,M,Titular,Ruiz,Montenegro,...,Jose,Eduardo,Jose Eduardo,1967-08-17 00:00:00,17,8,1967,1967-08-17,eruizmontenegro@gmail.com,NaN
5,Pesco Seguridad y Servicios S.A.,10411651,5,NaN,NaN,10411651-5,M,Titular,Piraino,Del,...,Pino,Carlo,Pino Carlo,1976-07-11 00:00:00,11,7,1976,1976-07-11,cfpiraino@gmail.com,NaN
6,Pesco Seguridad y Servicios S.A.,10698849,8,NaN,NaN,10698849-8,M,Titular,Arcos,Hernandez,...,Rodrigo,Rolando,Rodrigo Rolando,1970-02-12 00:00:00,12,2,1970,1970-02-12,rodrigo.arcoshernandez@gmail.com,NaN
7,Pesco Seguridad y Servicios S.A.,11548014,6,NaN,NaN,11548014-6,M,Titular,Nuñez,Villarroel,...,Paulo,Cesar,Paulo Cesar,1970-10-27 00:00:00,27,10,1970,1970-10-27,paulocnunezv@gmail.com,982537643.0
8,Pesco Seguridad y Servicios S.A.,11728454,9,NaN,NaN,11728454-9,M,Titular,Gonzalez,Tapia,...,Luis,Sandro,Luis Sandro,1971-04-21 00:00:00,21,4,1971,1971-04-21,luissandrogt21@gmail.com,968036646.0
9,Pesco Seguridad y Servicios S.A.,12033441,7,NaN,NaN,12033441-7,M,Titular,Zapata,Marquez,...,Rodrigo,Alberto,Rodrigo Alberto,1968-09-09 00:00:00,9,9,1968,1968-09-09,rodrigoalberto.zapata.m@gmail.com,950884769.0


In [27]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [28]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,Empresa,Rut Titular,DV,Rut carga,Dv carga,rut1,sexo,Parentesco,Apellido Paterno,Apellido Mateno,...,Nombre 2,nombres,naxxxcimiento,Columna3,Columna4,Columna1,nacimiento,email,celular,rut
0,Pesco Seguridad y Servicios S.A.,8650247,k,NaN,NaN,8650247-k,M,Titular,Villalobos,Riquelme,...,Antonio,Marco Antonio,1964-02-26 00:00:00,26,2,1964,1964-02-26,bomberovillalobos@hotmail.com,948941996.0,8650247k
1,Pesco Seguridad y Servicios S.A.,8849139,4,NaN,NaN,8849139-4,M,Titular,Barahona,Lobos,...,Miguel,Patricio Miguel,1963-05-03 00:00:00,3,5,1963,1963-05-03,barahona.patricio@gmail.com,957288339.0,88491394
2,Pesco Seguridad y Servicios S.A.,9120472,k,NaN,NaN,9120472-k,M,Titular,Soza,Romero,...,Alejandro,Jose Alejandro,1962-01-10 00:00:00,10,1,1962,1962-01-10,josesozar@gmail.com,NaN,9120472k
3,Pesco Seguridad y Servicios S.A.,9616064,k,NaN,NaN,9616064-k,M,Titular,Perez,Anker,...,Joaquin,Jose Joaquin,1969-07-08 00:00:00,8,7,1969,1969-07-08,jperezanker@gmail.com,987740031.0,9616064k
4,Pesco Seguridad y Servicios S.A.,9906103,0,NaN,NaN,9906103-0,M,Titular,Ruiz,Montenegro,...,Eduardo,Jose Eduardo,1967-08-17 00:00:00,17,8,1967,1967-08-17,eruizmontenegro@gmail.com,NaN,99061030


In [29]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head() 

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9824\485331713.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,Empresa,Rut Titular,DV,Rut carga,Dv carga,rut1,sexo,Parentesco,Apellido Paterno,Apellido Mateno,...,Nombre 2,nombres,naxxxcimiento,Columna3,Columna4,Columna1,nacimiento,email,celular,rut
0,Pesco Seguridad y Servicios S.A.,8650247,k,NaN,NaN,8650247-k,M,Titular,Villalobos,Riquelme,...,Antonio,Marco Antonio,1964-02-26 00:00:00,26,2,1964,1964-02-26,bomberovillalobos@hotmail.com,948941996.0,8650247K
1,Pesco Seguridad y Servicios S.A.,8849139,4,NaN,NaN,8849139-4,M,Titular,Barahona,Lobos,...,Miguel,Patricio Miguel,1963-05-03 00:00:00,3,5,1963,1963-05-03,barahona.patricio@gmail.com,957288339.0,88491394
2,Pesco Seguridad y Servicios S.A.,9120472,k,NaN,NaN,9120472-k,M,Titular,Soza,Romero,...,Alejandro,Jose Alejandro,1962-01-10 00:00:00,10,1,1962,1962-01-10,josesozar@gmail.com,NaN,9120472K
3,Pesco Seguridad y Servicios S.A.,9616064,k,NaN,NaN,9616064-k,M,Titular,Perez,Anker,...,Joaquin,Jose Joaquin,1969-07-08 00:00:00,8,7,1969,1969-07-08,jperezanker@gmail.com,987740031.0,9616064K
4,Pesco Seguridad y Servicios S.A.,9906103,0,NaN,NaN,9906103-0,M,Titular,Ruiz,Montenegro,...,Eduardo,Jose Eduardo,1967-08-17 00:00:00,17,8,1967,1967-08-17,eruizmontenegro@gmail.com,NaN,99061030


In [30]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9824\2118328728.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_9824\2118328728.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [31]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(235, 32)

In [32]:
Pacientes_con_id.head()

,Empresa,Rut Titular,DV,Rut carga,Dv carga,rut1,sexo,Parentesco,Apellido Paterno,Apellido Mateno,...,RutEstaoNo,Id Account Paciente,Rut,Nombre_y,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,Pesco Seguridad y Servicios S.A.,8650247,k,NaN,NaN,8650247-k,M,Titular,Villalobos,Riquelme,...,True,0018c00002eislCAAQ,8650247K,Marco Antonio Villalobos Riquelme,RUT,None,None,None,Masculino,None
1,Pesco Seguridad y Servicios S.A.,8849139,4,NaN,NaN,8849139-4,M,Titular,Barahona,Lobos,...,True,0018c00002cVuubAAC,88491394,Patricio Miguel Barahona Lobos,RUT,None,None,None,Hombre,None
2,Pesco Seguridad y Servicios S.A.,9120472,k,NaN,NaN,9120472-k,M,Titular,Soza,Romero,...,True,0018c00002cVnDxAAK,9120472K,Jose Alejandro Soza Romero,RUT,None,None,None,Hombre,None
3,Pesco Seguridad y Servicios S.A.,9616064,k,NaN,NaN,9616064-k,M,Titular,Perez,Anker,...,True,0018c00002cVnE7AAK,9616064K,Jose Joaquin Perez Anker,RUT,None,None,None,Hombre,None
4,Pesco Seguridad y Servicios S.A.,9906103,0,NaN,NaN,9906103-0,M,Titular,Ruiz,Montenegro,...,True,0018c00002cVnDyAAK,99061030,Jose Eduardo Ruiz Montenegro,RUT,None,None,None,Hombre,None


In [33]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Pacientes_con_id["rut"].astype(str).str.upper()
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        'Varón':'Hombre'      
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = Pacientes_con_id["celular"].apply(str).str.strip().replace("+","")
    Subir_pacientes['Phone'] = Subir_pacientes['Phone'].str.title().replace("NaN", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"




if Pais == 'Chile':        
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtnEAE'
else:    
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtsEAE'


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,Phone,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
10,False,Giovanni Francisco,Barrios Rojas,12513324K,gbarriosrojas1973@gmail.com,1973-10-08,Hombre,Nan,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
14,False,Hector,Velozo Real,128809643,hector.velozo@gmail.com,1975-09-13,Hombre,978456948.0,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
31,False,Fernando Antonio,Mella Fuenzalida,153744882,fdo.mella@gmail.com,1982-11-11,Hombre,985058218.0,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
32,False,Cristian Andres,Gonzalez Sanhueza,153761507,sanhueza07@gmail.com,1983-07-06,Hombre,Nan,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
35,False,Felix Eduardo,San Cristobal San Cristobal,159712451,sancristobal.felix@gmail.com,1985-01-27,Hombre,974958595.0,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE


In [34]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "SURA": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [35]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
606015,a1Y8c00000DocuVEAR,0018c00002l67UpAAI,a1W8c0000083wi3EAA,Grupo PSG,0018c00002jp25SAAQ,Grupo PSG SPA,Sin Holding Asociado,Grupo PSG,Grupo PSG SPA,Sin Póliza Asociada,EMP10370,A-9927


In [36]:
campaña_seleccionada.shape

(1, 12)

In [37]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [38]:
campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
campaña_en_pacientes = campaña_en_pacientes.rename(columns={'Programas_por_Empresas__c':'programas por empresa'})
campaña_en_pacientes['Paciente__c']=campaña_en_pacientes['Id Account Paciente']
campaña_en_pacientes['Programas_por_Empresas__c']=campaña_en_pacientes['programas por empresa']
campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [39]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)
